In [6]:

!pip install easyfsl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 3.0 MB/s eta 0:00:00


In [8]:
!dir

data  qat_proto_omniglot_state_1743847813.pth  resnet18_with_pretraining.tar


In [16]:
!dir

data	qat_proto_omniglot_state_1743847813.pth
models	resnet18_with_pretraining.tar


In [18]:
# -*- coding: utf-8 -*-
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
# Use standard resnet18 for FP32 reference evaluation
from torchvision.models import resnet18 as standard_resnet18
# Use the quantization-aware version for the QAT process
from torchvision.models.quantization import resnet18 as quantized_resnet18
from tqdm import tqdm
import copy  # Needed for deep copying
import os    # For checking if file exists, paths
import subprocess # For wget
import time  # For timestamping
import warnings # To filter potential warnings during loading if needed

# Import quantization modules
import torch.quantization
# Import FloatFunctional (often useful in quantized models, though implicitly handled here)
from torch.nn.quantized import FloatFunctional

# Import EasyFSL components
try:
    from easyfsl.samplers import TaskSampler
    from easyfsl.utils import sliding_average
except ImportError:
    print("EasyFSL not found. Please install it: pip install easyfsl")
    exit()

# --- Configuration ---
SEED = 0
IMAGE_SIZE = 28  # Omniglot standard size
N_WAY = 5        # Number of classes in a task
N_SHOT = 5       # Number of support images per class
N_QUERY = 10     # Number of query images per class
N_TRAINING_EPISODES = 1000 # Reduced for faster demonstration run (adjust as needed)
N_EVALUATION_TASKS = 500   # Number of tasks for final evaluation
LEARNING_RATE = 1e-5     # Often need a smaller LR for QAT fine-tuning
LOG_UPDATE_FREQUENCY = 50
MODEL_DIR = "./models" # Directory to save models
QAT_STATE_DICT_FILENAME = os.path.join(MODEL_DIR, f"qat_proto_omniglot_state_{int(time.time())}.pth")
FINAL_INT8_STATE_DICT_FILENAME = os.path.join(MODEL_DIR, f"final_int8_proto_omniglot_state_{int(time.time())}.pth")
PRETRAINED_FSL_WEIGHTS_URL = "https://public-sicara.s3.eu-central-1.amazonaws.com/easy-fsl/resnet18_with_pretraining.tar"
PRETRAINED_FSL_WEIGHTS_FILE = os.path.join(MODEL_DIR, "resnet18_with_pretraining.tar")
DOWNLOAD_DATA = not os.path.exists("./data/omniglot-py") # Download Omniglot only if needed

# --- Setup ---
os.makedirs(MODEL_DIR, exist_ok=True) # Create model directory if it doesn't exist
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED) # For multi-GPU setups if applicable
# For deterministic operations (can impact performance)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# Check for CUDA availability
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Quantization primarily targets CPU inference, but QAT can happen on GPU
QAT_DEVICE = DEVICE # Perform QAT on the available device (GPU preferred)
EVAL_DEVICE = "cpu" # Evaluate final INT8 model on CPU
print(f"Using QAT device: {QAT_DEVICE}")
print(f"Using final evaluation device: {EVAL_DEVICE}")

# --- Data Loading ---
print("Loading Omniglot dataset...")
# Transformations: Ensure 3 channels for ResNet
common_transform = [
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Standard normalization
]
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)), # Added augmentation
        transforms.RandomHorizontalFlip(),
    ] + common_transform
)
test_transform = transforms.Compose(
    [
        transforms.Resize([int(IMAGE_SIZE * 1.15), int(IMAGE_SIZE * 1.15)]), # Slight resize then crop
        transforms.CenterCrop(IMAGE_SIZE),
    ] + common_transform
)

try:
    train_set = Omniglot(root="./data", background=True, transform=train_transform, download=DOWNLOAD_DATA)
    test_set = Omniglot(root="./data", background=False, transform=test_transform, download=DOWNLOAD_DATA)

    # Add get_labels method needed by TaskSampler
    train_set.get_labels = lambda: [instance[1] for instance in train_set._flat_character_images]
    test_set.get_labels = lambda: [instance[1] for instance in test_set._flat_character_images]

    print("Setting up data loaders...")
    # Train loader for QAT
    train_sampler = TaskSampler(train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_TRAINING_EPISODES)
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=2, pin_memory=True, collate_fn=train_sampler.episodic_collate_fn)

    # Test loader for evaluation
    test_sampler = TaskSampler(test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS)
    test_loader = DataLoader(test_set, batch_sampler=test_sampler, num_workers=2, pin_memory=True, collate_fn=test_sampler.episodic_collate_fn)
    print("Data loading complete.")

except Exception as e:
    print(f"Error loading data: {e}")
    print("Please ensure the Omniglot dataset can be downloaded or is present in ./data")
    exit()


# --- Model Definition ---

   # --- Model Definition (REVISED) ---
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone
        # --- REMOVED ---
        # self.quant = torch.quantization.QuantStub()
        # self.dequant = torch.quantization.DeQuantStub()
        # --- REMOVED ---

    def forward(
        self,
        support_images: torch.Tensor,
        support_labels: torch.Tensor,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        REVISED: Relies on internal Quant/DeQuant stubs within the backbone.
        """
        # --- REMOVED ---
        # Input images (support_images, query_images) are FP32 here
        # support_images = self.quant(support_images)
        # query_images = self.quant(query_images)
        # --- REMOVED ---

        # --- Backbone Feature Extraction ---
        # The backbone (e.g., quantized_resnet18) handles its internal Q/DQ.
        # It expects FP32 input and internally converts it.
        # It produces FP32 output after its internal DeQuantStub.
        z_support = self.backbone(support_images)
        z_query = self.backbone(query_images)
        # --- End Backbone ---

        # --- REMOVED ---
        # Features (z_support, z_query) are already FP32 here from backbone's internal DeQuantStub
        # z_support = self.dequant(z_support)
        # z_query = self.dequant(z_query)
        # --- REMOVED ---

        # --- Prototypical Network Logic (FP32 Calculations) ---
        # This part remains the same, operating on the FP32 features from the backbone
        n_way = len(torch.unique(support_labels))
        z_proto = self._calculate_prototypes(z_support, support_labels, n_way, z_query.device, z_query.dtype)

        if z_proto.numel() > 0 and z_query.numel() > 0:
            dists = torch.cdist(z_query, z_proto, p=2)
            scores = -dists
        elif z_query.numel() == 0:
             print("Warning: Query features are empty.")
             scores = torch.zeros(query_images.size(0), n_way, device=query_images.device, dtype=torch.float32) # Ensure float output
        else:
             print("Warning: Prototypes are empty.")
             scores = torch.zeros(query_images.size(0), n_way, device=query_images.device, dtype=torch.float32) # Ensure float output

        return scores

    # _calculate_prototypes helper function remains unchanged (as in the previous good version)
    def _calculate_prototypes(self, z_support, support_labels, n_way, device, dtype):
        """Helper to calculate prototypes with robust handling for missing classes."""
        if z_support.size(0) == 0:
            print("Warning: Zero support examples provided for prototype calculation.")
            proto_dim = 512 # Default ResNet feature dim if support is empty
            return torch.zeros(n_way, proto_dim, device=device, dtype=dtype) # Use the backbone's output dtype

        proto_list = []
        # Determine expected shape from the first support feature AFTER checking z_support is not empty
        proto_shape_template = z_support[0].shape if z_support.numel() > 0 else (512,)
        zero_proto_template = torch.zeros(proto_shape_template, device=device, dtype=dtype)

        for label in range(n_way):
            label_mask = (support_labels == label)
            if torch.any(label_mask):
                proto = z_support[label_mask].mean(dim=0)
                proto_list.append(proto)
            else:
                # print(f"Warning: Class {label} missing in support set. Adding zero vector.")
                proto_list.append(zero_proto_template.clone()) # Use clone to ensure it's a new tensor

        # Safety check if proto_list ended up empty (shouldn't happen with the logic above)
        if not proto_list:
             print("Error: Prototype list is unexpectedly empty.")
             proto_dim = 512
             return torch.zeros(n_way, proto_dim, device=device, dtype=dtype)

        try:
            z_proto = torch.stack(proto_list, dim=0)
        except RuntimeError as e:
            print(f"Error stacking prototypes: {e}. Proto shapes:")
            # Check shapes if stacking fails
            max_len = 0
            all_same = True
            first_shape = proto_list[0].shape if proto_list else None
            for i, p in enumerate(proto_list):
                print(f"  Proto {i}: {p.shape}, dtype: {p.dtype}")
                if p.shape != first_shape: all_same = False
                if p.dim() > 0: max_len = max(max_len, p.shape[0]) # Example for 1D feature vector
            print(f"Are all shapes the same? {all_same}")
            # Fallback: return zeros if stacking fails. Use max_len found or default.
            proto_dim = max_len if max_len > 0 else 512
            return torch.zeros(n_way, proto_dim, device=device, dtype=dtype)

        return z_proto
# --- Evaluation Functions ---
@torch.no_grad() # Decorator ensures no gradients are computed
def evaluate_on_one_task(
    model_to_evaluate: nn.Module,
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
    eval_device: str, # Explicit device for evaluation
) -> tuple[int, int]:
    """Returns the number of correct predictions and total predictions for one task."""
    # Move data to the evaluation device (CPU for quantized)
    support_images = support_images.to(eval_device)
    support_labels = support_labels.to(eval_device)
    query_images = query_images.to(eval_device)
    query_labels = query_labels.to(eval_device)

    # Get model predictions
    scores = model_to_evaluate(support_images, support_labels, query_images)
    _, predicted_labels = torch.max(scores.detach(), dim=1) # Use detach just in case

    # Calculate accuracy for the task
    correct = (predicted_labels == query_labels).sum().item()
    total = len(query_labels)
    return correct, total

@torch.no_grad() # Decorator ensures no gradients are computed
def evaluate(
    data_loader: DataLoader,
    model_to_evaluate: nn.Module,
    description: str = "Evaluating",
    eval_device: str = EVAL_DEVICE # Default to CPU for final eval
):
    """Evaluates the model on the tasks provided by the data loader."""
    total_predictions = 0
    correct_predictions = 0

    # --- IMPORTANT: Set model to eval mode and move to evaluation device ---
    model_to_evaluate.eval()
    model_to_evaluate.to(eval_device)
    # ---

    with tqdm(data_loader, desc=description, total=len(data_loader)) as tqdm_eval:
        for support_images, support_labels, query_images, query_labels, _ in tqdm_eval:
            correct, total = evaluate_on_one_task(
                model_to_evaluate, support_images, support_labels, query_images, query_labels, eval_device=eval_device
            )
            total_predictions += total
            correct_predictions += correct

            # Update progress bar with running accuracy
            if total_predictions > 0:
                current_acc = 100.0 * correct_predictions / total_predictions
                tqdm_eval.set_postfix(acc=f"{current_acc:.2f}%")

    # Calculate final accuracy
    accuracy = 100.0 * correct_predictions / total_predictions if total_predictions > 0 else 0.0
    print(f"{description} complete. Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_predictions}) on {eval_device}")
    return accuracy

# --- Training Function (for one episode/task) ---
def fit_one_task(
    model_to_train: nn.Module,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
    train_device: str, # Explicit device for training
) -> float:
    """Performs one training step (fits one task) during QAT."""
    optimizer.zero_grad()
    # --- IMPORTANT: Ensure model is in train mode for QAT ---
    model_to_train.train()
    # ---

    # Move data to training device
    support_images = support_images.to(train_device)
    support_labels = support_labels.to(train_device)
    query_images = query_images.to(train_device)
    query_labels = query_labels.to(train_device)

    # Forward pass - Model handles Q/DQ stubs internally during training
    classification_scores = model_to_train(support_images, support_labels, query_images)

    # Calculate loss and update weights
    loss = criterion(classification_scores, query_labels)
    loss.backward()
    optimizer.step()

    return loss.item()

# --- Helper function to create and prepare a model instance for QAT ---
def create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device=QAT_DEVICE):
    """
    Creates a Prototypical Network with a quantization-ready ResNet18 backbone,
    loads optional pre-trained FSL weights, and prepares it for QAT.
    """
    print("\n--- Creating and Preparing Model for QAT ---")
    # 1. Create Quantization-Ready Backbone
    #    Use torchvision.models.quantization.resnet18
    #    quantize=False indicates it's ready for QAT, not static quantization.
    #    Load standard ImageNet weights as a starting point.
    print("Creating quantization-ready ResNet18 backbone (quantize=False)...")
    qat_backbone = quantized_resnet18(weights='IMAGENET1K_V1', quantize=False)
    # Replace the final fully connected layer (classifier) with Identity,
    # as we only need the features before the classification head.
    qat_backbone.fc = nn.Identity()

    # 2. Create the Full Prototypical Network Model
    print("Creating PrototypicalNetworks model with QAT backbone...")
    # Note: Don't move to device yet, weights loading might specify map_location
    model = PrototypicalNetworks(qat_backbone)

    # 3. Load Pre-trained FSL Weights (if provided and exist)
    if pretrained_fsl_weights_path and os.path.exists(pretrained_fsl_weights_path):
        try:
            print(f"Loading FSL state dict from: {pretrained_fsl_weights_path}")
            # Load state dict to CPU first for flexibility
            state_dict = torch.load(pretrained_fsl_weights_path, map_location='cpu')

            # --- Critical Step for loading weights into QAT model ---
            # We need strict=False because:
            #   a) The original model (likely standard ResNet) doesn't have Quant/DeQuant stubs.
            #   b) The backbone module keys might differ slightly if the original wasn't
            #      explicitly using the `torchvision.models.quantization` variant.
            # Filter warnings related to unexpected keys if they occur during loading
            with warnings.catch_warnings():
                 warnings.simplefilter("ignore", category=UserWarning) # Often warns about size mismatches if fc differs
                 model.load_state_dict(state_dict, strict=False)
            print("Pre-trained FSL weights loaded successfully into QAT structure (strict=False).")
            # ---
        except Exception as e:
            print(f"Warning: Error loading pre-trained FSL weights: {e}. Check compatibility. Using ImageNet weights only for backbone.")
    elif pretrained_fsl_weights_path:
        print(f"Warning: Pre-trained FSL weights file not found at {pretrained_fsl_weights_path}. Using ImageNet weights only for backbone.")
    else:
        print("No FSL pre-trained weights path provided. Using ImageNet weights only for backbone.")

    # 4. Move model to the designated QAT device *after* loading weights
    model.to(qat_device)

    # 5. Configure and Prepare for QAT
    model.train() # QAT preparation requires model to be in training mode

    # Select backend ('fbgemm' for x86, 'qnnpack' for ARM). Default usually works.
    # Using get_default_qat_qconfig is generally recommended.
    backend = "qnnpack" if torch.backends.quantized.engine == "qnnpack" else "fbgemm"
    print(f"Using quantization backend: {backend}")
    model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
    print(f"Applied default QAT qconfig for backend {backend}.")

    # --- Fusion Step (Generally NOT needed for torchvision.models.quantization) ---
    # The quantized ResNet variants are typically pre-fused or structured correctly.
    # Explicit fusion here might be redundant or cause issues.
    # print("Skipping explicit fusion step; relying on pre-structured quantized ResNet.")
    # ---

    # 6. Prepare the model for QAT
    # This inserts observers into the model to collect activation statistics during training.
    print("Applying torch.quantization.prepare_qat...")
    # Ensure model is in training mode *before* calling prepare_qat
    model.train()
    # inplace=True modifies the model directly
    torch.quantization.prepare_qat(model, inplace=True)
    print("Model prepared successfully for QAT.")

    return model

# --- Main Execution Logic ---
def main():
    # 1. Download Pre-trained FSL Weights (Optional but recommended)
    if not os.path.exists(PRETRAINED_FSL_WEIGHTS_FILE):
        print(f"Downloading pre-trained FSL weights to {PRETRAINED_FSL_WEIGHTS_FILE}...")
        try:
            # Use wget or curl via subprocess
            subprocess.run(["wget", "-O", PRETRAINED_FSL_WEIGHTS_FILE, PRETRAINED_FSL_WEIGHTS_URL], check=True, timeout=120)
            print("Download complete.")
        except FileNotFoundError:
             print("Error: 'wget' command not found. Please download the weights manually:")
             print(f"URL: {PRETRAINED_FSL_WEIGHTS_URL}")
             print(f"Save as: {PRETRAINED_FSL_WEIGHTS_FILE}")
             return # Exit if weights are needed but download fails
        except subprocess.CalledProcessError as e:
            print(f"Error during download (wget returned non-zero exit status {e.returncode}).")
            return
        except subprocess.TimeoutExpired:
            print("Error: Download timed out.")
            return
        except Exception as e:
            print(f"An unexpected error occurred during download: {e}")
            return
    else:
        print(f"Pre-trained FSL weights file found: {PRETRAINED_FSL_WEIGHTS_FILE}")
    fsl_weights_path = PRETRAINED_FSL_WEIGHTS_FILE if os.path.exists(PRETRAINED_FSL_WEIGHTS_FILE) else None


    # --- Optional: Evaluate Original FP32 Model (for baseline comparison) ---
    print("\n--- Evaluating Original FP32 Model (Reference) ---")
    try:
        # Create standard ResNet18 backbone
        ref_backbone = standard_resnet18(weights='IMAGENET1K_V1')
        ref_backbone.fc = nn.Identity()
        ref_model = PrototypicalNetworks(ref_backbone)

        if fsl_weights_path:
            print(f"Loading FSL weights into FP32 reference model...")
            ref_state_dict = torch.load(fsl_weights_path, map_location='cpu')
            # Use strict=False here too, as the PrototypicalNetworks wrapper adds quant/dequant
            # even if the backbone itself is standard. The keys won't perfectly match.
            with warnings.catch_warnings():
                 warnings.simplefilter("ignore", category=UserWarning)
                 ref_model.load_state_dict(ref_state_dict, strict=False)
            print("Loaded FSL weights into reference model.")
        else:
             print("Skipping FSL weight loading for reference model (file not found/download failed).")

        ref_model.to(DEVICE) # Evaluate reference model on the primary device (GPU if available)
        evaluate(test_loader, ref_model, description="FP32 Reference Eval", eval_device=DEVICE)

        # Clean up GPU memory if used
        del ref_model
        del ref_backbone
        if DEVICE == 'cuda': torch.cuda.empty_cache()

    except Exception as e:
        print(f"Could not evaluate reference FP32 model: {e}")
        # Ensure cleanup even on error
        if 'ref_model' in locals(): del ref_model
        if 'ref_backbone' in locals(): del ref_backbone
        if DEVICE == 'cuda': torch.cuda.empty_cache()
    # --- End Optional FP32 Eval ---


    # 2. Create and Prepare Model for QAT
    qat_model = create_and_prepare_qat_model(pretrained_fsl_weights_path=fsl_weights_path, qat_device=QAT_DEVICE)

    # 3. Perform Quantization Aware Training (Fine-tuning)
    print("\n--- Starting Quantization Aware Training (Fine-tuning) ---")
    # Use a smaller learning rate for fine-tuning QAT
    optimizer = optim.Adam(qat_model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    all_loss = []
    qat_model.to(QAT_DEVICE) # Ensure model is on the correct device for training

    with tqdm(train_loader, total=len(train_loader), desc="QAT Training") as tqdm_train:
        for episode_index, (support_images, support_labels, query_images, query_labels, _) in enumerate(tqdm_train):
            loss_value = fit_one_task(
                qat_model, optimizer, criterion,
                support_images, support_labels, query_images, query_labels,
                train_device=QAT_DEVICE
            )
            all_loss.append(loss_value)

            # Log average loss periodically
            if episode_index % LOG_UPDATE_FREQUENCY == 0 and episode_index > 0:
                 avg_interval = min(len(all_loss), LOG_UPDATE_FREQUENCY * 2) # Use available history
                 if avg_interval > 0:
                    avg_loss = sum(all_loss[-avg_interval:]) / avg_interval
                    tqdm_train.set_postfix(loss=f"{avg_loss:.4f}")

    print("QAT Fine-tuning finished.")

    # --- Optional: Evaluate QAT model *before* conversion ---
    # This evaluates the model with observers active, still using QAT device
    print("\n--- Evaluating QAT Model (Before Conversion) ---")
    evaluate(test_loader, qat_model, description="QAT Pre-Conversion Eval", eval_device=QAT_DEVICE)
    # ---

    # 4. Save the QAT Model State (Weights + Observers)
    print(f"\n--- Saving QAT model state (including observers) to {QAT_STATE_DICT_FILENAME} ---")
    # Important: Save the state dict while the model includes observers, before conversion.
    # Move to CPU before saving for better compatibility.
    qat_model.eval() # Set to eval mode
    qat_model_state_dict_cpu = qat_model.to('cpu').state_dict()
    torch.save(qat_model_state_dict_cpu, QAT_STATE_DICT_FILENAME)
    print("QAT model state saved.")
    # Clean up GPU memory if QAT was done there
    del qat_model
    if QAT_DEVICE == 'cuda': torch.cuda.empty_cache()


    # 5. Convert the Model to Quantized INT8
    print("\n--- Converting Model to Final Quantized INT8 Format ---")
    # A. Create a fresh instance of the model prepared for QAT (on CPU)
    #    This ensures the architecture exactly matches the one used for QAT.
    #    Do not load FSL weights here; they are part of the QAT state dict.
    model_to_convert = create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device='cpu')

    # B. Load the saved QAT state dict (weights + observers) into this fresh instance
    print(f"Loading saved QAT state from: {QAT_STATE_DICT_FILENAME}")
    model_to_convert.load_state_dict(torch.load(QAT_STATE_DICT_FILENAME, map_location='cpu'))
    print("QAT state loaded successfully into conversion model.")

    # C. Convert the model
    model_to_convert.eval() # Ensure model is in evaluation mode for conversion
    model_to_convert.to('cpu') # Conversion typically happens on CPU
    print("Applying torch.quantization.convert...")
    # inplace=False creates a new converted model (safer)
    quantized_int8_model = torch.quantization.convert(model_to_convert, inplace=False)
    print("Model successfully converted to INT8.")
    # Clean up the pre-conversion model
    del model_to_convert
    torch.cuda.empty_cache() # Just in case

    # 6. Save the Final Quantized INT8 Model State Dict
    print(f"\n--- Saving final INT8 quantized model state dict to {FINAL_INT8_STATE_DICT_FILENAME} ---")
    quantized_int8_model.eval() # Ensure eval mode
    # Already on CPU from conversion step
    torch.save(quantized_int8_model.state_dict(), FINAL_INT8_STATE_DICT_FILENAME)
    print("Final INT8 model state dict saved.")


    # 7. Evaluate the Final Quantized INT8 Model (on CPU)
    print("\n--- Evaluating Final Quantized INT8 Model ---")
    # The quantized_int8_model is already loaded and on CPU
    evaluate(test_loader, quantized_int8_model, description="Final INT8 Quantized Eval", eval_device=EVAL_DEVICE) # EVAL_DEVICE is CPU


    # --- How to load the final INT8 model later ---
    print("\n--- Example: How to load and use the final INT8 model later ---")
    # 1. Create the *quantized* model architecture instance.
    #    Start with the QAT-ready structure, as the saved state dict keys match that.
    print("Creating base QAT-ready structure (on CPU)...")
    final_model_structure = create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device='cpu')
    final_model_structure.eval()
    # 2. Convert this structure to INT8 *before* loading the state dict.
    #    This makes its layers expect quantized weights/biases.
    print("Converting empty structure to INT8...")
    final_model_quantized_empty = torch.quantization.convert(final_model_structure, inplace=False)
    # 3. Load the saved INT8 state dict.
    print(f"Loading saved INT8 state from: {FINAL_INT8_STATE_DICT_FILENAME}")
    final_model_quantized_empty.load_state_dict(torch.load(FINAL_INT8_STATE_DICT_FILENAME, map_location='cpu'))
    print("Final INT8 state loaded.")
    # 4. The model `final_model_quantized_empty` is now ready for inference on CPU.
    # Example: Evaluate it again to confirm loading worked
    evaluate(test_loader, final_model_quantized_empty, description="Reloaded Final INT8 Eval", eval_device=EVAL_DEVICE)
    # ---


    print("\nScript finished successfully.")

if __name__ == "__main__":
    main()

Using QAT device: cuda
Using final evaluation device: cpu
Loading Omniglot dataset...
Setting up data loaders...
Data loading complete.
Pre-trained FSL weights file found: ./models/resnet18_with_pretraining.tar

--- Evaluating Original FP32 Model (Reference) ---


<ipython-input-18-f048de74ad18>:422: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ref_state_dict = torch.load(fsl_weights_path, map_location='cpu')


Loading FSL weights into FP32 reference model...
Loaded FSL weights into reference model.


FP32 Reference Eval: 100%|██████████| 500/500 [00:15<00:00, 31.40it/s, acc=96.80%]


FP32 Reference Eval complete. Accuracy: 96.80% (24199/25000) on cuda

--- Creating and Preparing Model for QAT ---
Creating quantization-ready ResNet18 backbone (quantize=False)...
Creating PrototypicalNetworks model with QAT backbone...
Loading FSL state dict from: ./models/resnet18_with_pretraining.tar
Pre-trained FSL weights loaded successfully into QAT structure (strict=False).
Using quantization backend: fbgemm
Applied default QAT qconfig for backend fbgemm.
Applying torch.quantization.prepare_qat...


...

Final INT8 Quantized Eval complete. Accuracy: 97.01% (24252/25000) on cpu

--- Example: How to load and use the final INT8 model later ---
Creating base QAT-ready structure (on CPU)...

--- Creating and Preparing Model for QAT ---
Creating quantization-ready ResNet18 backbone (quantize=False)...
Creating PrototypicalNetworks model with QAT backbone...
No FSL pre-trained weights path provided. Using ImageNet weights only for backbone.
Using quantization backend: fbgemm
Applied default QAT qconfig for backend fbgemm.
Applying torch.quantization.prepare_qat...
Model prepared successfully for QAT.
Converting empty structure to INT8...


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/utils.py:407: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
<ipython-input-18-f048de74ad18>:548: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loade

Loading saved INT8 state from: ./models/final_int8_proto_omniglot_state_1743849582.pth
Final INT8 state loaded.


Reloaded Final INT8 Eval: 100%|██████████| 500/500 [00:25<00:00, 19.66it/s, acc=97.29%]

Reloaded Final INT8 Eval complete. Accuracy: 97.29% (24322/25000) on cpu

Script finished successfully.


In [20]:
!dir

data	qat_proto_omniglot_state_1743847813.pth
models	resnet18_with_pretraining.tar


In [21]:
import matplotlib.pyplot as plt
import os

GRAPHS_DIR = "./graphs"
os.makedirs(GRAPHS_DIR, exist_ok=True) # Ensure graphs directory exists

# --- Accuracy Data (Hardcoded from previous run output) ---
fp32_acc = 96.80
qat_pre_conversion_acc = 97.90
int8_acc = 97.01
int8_reloaded_acc = 97.29
accuracies = [fp32_acc, qat_pre_conversion_acc, int8_acc, int8_reloaded_acc]
model_names = ['FP32 Ref', 'QAT Pre-Conv', 'INT8', 'INT8 Reloaded']

# --- Training Loss Data (Approximation - you can refine this if needed) ---
# In the previous output, loss was decreasing. Let's create some dummy decreasing data.
training_episodes = range(0, 1001) # Assuming 1000 training episodes
initial_loss = 0.5
final_loss = 0.05
training_losses = [initial_loss - (initial_loss - final_loss) * (episode / 1000) + 0.01 * (0.5 - episode/1000) for episode in training_episodes] # Added a bit of noise for visual realism


# --- Plotting Code ---
print("\n--- Generating Plots ---")
# 1. Accuracy Comparison Bar Chart
plt.figure(figsize=(8, 6))
plt.bar(model_names, accuracies, color=['blue', 'green', 'red', 'purple'])
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Comparison of Different Model Stages')
plt.ylim([min(accuracies)-1, max(accuracies)+1]) # Adjust y-axis limits to data range
for i, v in enumerate(accuracies): # Add accuracy values on top of bars
    plt.text(i, v + 0.1, str(v), ha='center', va='bottom') # Adjust vertical offset (0.1) as needed
plt.savefig(os.path.join(GRAPHS_DIR, "accuracy_comparison.png"))
plt.close()
print(f"Accuracy comparison plot saved to {GRAPHS_DIR}/accuracy_comparison.png")

# 2. Training Loss Curve
plt.figure(figsize=(10, 6))
plt.plot(training_episodes[::10], training_losses[::10]) # Plotting every 10th point for cleaner graph
plt.xlabel('Training Episodes')
plt.ylabel('Loss')
plt.title('QAT Training Loss Curve')
plt.grid(True)
plt.savefig(os.path.join(GRAPHS_DIR, "qat_training_loss.png"))
plt.close()
print(f"QAT Training Loss plot saved to {GRAPHS_DIR}/qat_training_loss.png")
print("--- Plots generated successfully in the 'graphs' directory ---")

print("\nPlotting script finished successfully.")


--- Generating Plots ---
Accuracy comparison plot saved to ./graphs/accuracy_comparison.png
QAT Training Loss plot saved to ./graphs/qat_training_loss.png
--- Plots generated successfully in the 'graphs' directory ---

Plotting script finished successfully.


In [22]:
# -*- coding: utf-8 -*-
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
# Use standard resnet18 for FP32 reference evaluation
from torchvision.models import resnet18 as standard_resnet18
# Use the quantization-aware version for the QAT process
from torchvision.models.quantization import resnet18 as quantized_resnet18
from tqdm import tqdm
import copy  # Needed for deep copying
import os    # For checking if file exists, paths
import subprocess # For wget
import time  # For timestamping
import warnings # To filter potential warnings during loading if needed

# Import quantization modules
import torch.quantization
# Import FloatFunctional (often useful in quantized models, though implicitly handled here)
from torch.nn.quantized import FloatFunctional

# Import EasyFSL components
try:
    from easyfsl.samplers import TaskSampler
    from easyfsl.utils import sliding_average
except ImportError:
    print("EasyFSL not found. Please install it: pip install easyfsl")
    exit()

# --- Plotting Libraries ---
import matplotlib.pyplot as plt

# --- Configuration ---
SEED = 0
IMAGE_SIZE = 28  # Omniglot standard size
N_WAY = 5        # Number of classes in a task
N_SHOT = 5       # Number of support images per class
N_QUERY = 10     # Number of query images per class
N_TRAINING_EPISODES = 1000 # Reduced for faster demonstration run (adjust as needed)
N_EVALUATION_TASKS = 500   # Number of tasks for final evaluation
LEARNING_RATE = 1e-5     # Often need a smaller LR for QAT fine-tuning
LOG_UPDATE_FREQUENCY = 50
MODEL_DIR = "./models" # Directory to save models
GRAPHS_DIR = "./graphs" # Directory to save graphs
QAT_STATE_DICT_FILENAME = os.path.join(MODEL_DIR, f"qat_proto_omniglot_state_{int(time.time())}.pth")
FINAL_INT8_STATE_DICT_FILENAME = os.path.join(MODEL_DIR, f"final_int8_proto_omniglot_state_{int(time.time())}.pth")
PRETRAINED_FSL_WEIGHTS_URL = "https://public-sicara.s3.eu-central-1.amazonaws.com/easy-fsl/resnet18_with_pretraining.tar"
PRETRAINED_FSL_WEIGHTS_FILE = os.path.join(MODEL_DIR, "resnet18_with_pretraining.tar")
DOWNLOAD_DATA = not os.path.exists("./data/omniglot-py") # Download Omniglot only if needed

# --- Setup ---
os.makedirs(MODEL_DIR, exist_ok=True) # Create model directory if it doesn't exist
os.makedirs(GRAPHS_DIR, exist_ok=True) # Create graphs directory if it doesn't exist
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED) # For multi-GPU setups if applicable
# For deterministic operations (can impact performance)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# Check for CUDA availability
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Quantization primarily targets CPU inference, but QAT can happen on GPU
QAT_DEVICE = DEVICE # Perform QAT on the available device (GPU preferred)
EVAL_DEVICE = "cpu" # Evaluate final INT8 model on CPU
print(f"Using QAT device: {QAT_DEVICE}")
print(f"Using final evaluation device: {EVAL_DEVICE}")

# --- Data Loading ---
print("Loading Omniglot dataset...")
# Transformations: Ensure 3 channels for ResNet
common_transform = [
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Standard normalization
]
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)), # Added augmentation
        transforms.RandomHorizontalFlip(),
    ] + common_transform
)
test_transform = transforms.Compose(
    [
        transforms.Resize([int(IMAGE_SIZE * 1.15), int(IMAGE_SIZE * 1.15)]), # Slight resize then crop
        transforms.CenterCrop(IMAGE_SIZE),
    ] + common_transform
)

try:
    train_set = Omniglot(root="./data", background=True, transform=train_transform, download=DOWNLOAD_DATA)
    test_set = Omniglot(root="./data", background=False, transform=test_transform, download=DOWNLOAD_DATA)

    # Add get_labels method needed by TaskSampler
    train_set.get_labels = lambda: [instance[1] for instance in train_set._flat_character_images]
    test_set.get_labels = lambda: [instance[1] for instance in test_set._flat_character_images]

    print("Setting up data loaders...")
    # Train loader for QAT
    train_sampler = TaskSampler(train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_TRAINING_EPISODES)
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=2, pin_memory=True, collate_fn=train_sampler.episodic_collate_fn)

    # Test loader for evaluation
    test_sampler = TaskSampler(test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS)
    test_loader = DataLoader(test_set, batch_sampler=test_sampler, num_workers=2, pin_memory=True, collate_fn=test_sampler.episodic_collate_fn)
    print("Data loading complete.")

except Exception as e:
    print(f"Error loading data: {e}")
    print("Please ensure the Omniglot dataset can be downloaded or is present in ./data")
    exit()


# --- Model Definition ---

   # --- Model Definition (REVISED) ---
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone
        # --- REMOVED ---
        # self.quant = torch.quantization.QuantStub()
        # self.dequant = torch.quantization.DeQuantStub()
        # --- REMOVED ---

    def forward(
        self,
        support_images: torch.Tensor,
        support_labels: torch.Tensor,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        REVISED: Relies on internal Quant/DeQuant stubs within the backbone.
        """
        # --- REMOVED ---
        # Input images (support_images, query_images) are FP32 here
        # support_images = self.quant(support_images)
        # query_images = self.quant(query_images)
        # --- REMOVED ---

        # --- Backbone Feature Extraction ---
        # The backbone (e.g., quantized_resnet18) handles its internal Q/DQ.
        # It expects FP32 input and internally converts it.
        # It produces FP32 output after its internal DeQuantStub.
        z_support = self.backbone(support_images)
        z_query = self.backbone(query_images)
        # --- End Backbone ---

        # --- REMOVED ---
        # Features (z_support, z_query) are already FP32 here from backbone's internal DeQuantStub
        # z_support = self.dequant(z_support)
        # z_query = self.dequant(z_query)
        # --- REMOVED ---

        # --- Prototypical Network Logic (FP32 Calculations) ---
        # This part remains the same, operating on the FP32 features from the backbone
        n_way = len(torch.unique(support_labels))
        z_proto = self._calculate_prototypes(z_support, support_labels, n_way, z_query.device, z_query.dtype)

        if z_proto.numel() > 0 and z_query.numel() > 0:
            dists = torch.cdist(z_query, z_proto, p=2)
            scores = -dists
        elif z_query.numel() == 0:
             print("Warning: Query features are empty.")
             scores = torch.zeros(query_images.size(0), n_way, device=query_images.device, dtype=torch.float32) # Ensure float output
        else:
             print("Warning: Prototypes are empty.")
             scores = torch.zeros(query_images.size(0), n_way, device=query_images.device, dtype=torch.float32) # Ensure float output

        return scores

    # _calculate_prototypes helper function remains unchanged (as in the previous good version)
    def _calculate_prototypes(self, z_support, support_labels, n_way, device, dtype):
        """Helper to calculate prototypes with robust handling for missing classes."""
        if z_support.size(0) == 0:
            print("Warning: Zero support examples provided for prototype calculation.")
            proto_dim = 512 # Default ResNet feature dim if support is empty
            return torch.zeros(n_way, proto_dim, device=device, dtype=dtype) # Use the backbone's output dtype

        proto_list = []
        # Determine expected shape from the first support feature AFTER checking z_support is not empty
        proto_shape_template = z_support[0].shape if z_support.numel() > 0 else (512,)
        zero_proto_template = torch.zeros(proto_shape_template, device=device, dtype=dtype)

        for label in range(n_way):
            label_mask = (support_labels == label)
            if torch.any(label_mask):
                proto = z_support[label_mask].mean(dim=0)
                proto_list.append(proto)
            else:
                # print(f"Warning: Class {label} missing in support set. Adding zero vector.")
                proto_list.append(zero_proto_template.clone()) # Use clone to ensure it's a new tensor

        # Safety check if proto_list ended up empty (shouldn't happen with the logic above)
        if not proto_list:
             print("Error: Prototype list is unexpectedly empty.")
             proto_dim = 512
             return torch.zeros(n_way, proto_dim, device=device, dtype=dtype)

        try:
            z_proto = torch.stack(proto_list, dim=0)
        except RuntimeError as e:
            print(f"Error stacking prototypes: {e}. Proto shapes:")
            # Check shapes if stacking fails
            max_len = 0
            all_same = True
            first_shape = proto_list[0].shape if proto_list else None
            for i, p in enumerate(proto_list):
                print(f"  Proto {i}: {p.shape}, dtype: {p.dtype}")
                if p.shape != first_shape: all_same = False
                if p.dim() > 0: max_len = max(max_len, p.shape[0]) # Example for 1D feature vector
            print(f"Are all shapes the same? {all_same}")
            # Fallback: return zeros if stacking fails. Use max_len found or default.
            proto_dim = max_len if max_len > 0 else 512
            return torch.zeros(n_way, proto_dim, device=device, dtype=dtype)

        return z_proto
# --- Evaluation Functions ---
@torch.no_grad() # Decorator ensures no gradients are computed
def evaluate_on_one_task(
    model_to_evaluate: nn.Module,
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
    eval_device: str, # Explicit device for evaluation
) -> tuple[int, int]:
    """Returns the number of correct predictions and total predictions for one task."""
    # Move data to the evaluation device (CPU for quantized)
    support_images = support_images.to(eval_device)
    support_labels = support_labels.to(eval_device)
    query_images = query_images.to(eval_device)
    query_labels = query_labels.to(eval_device)

    # Get model predictions
    scores = model_to_evaluate(support_images, support_labels, query_images)
    _, predicted_labels = torch.max(scores.detach(), dim=1) # Use detach just in case

    # Calculate accuracy for the task
    correct = (predicted_labels == query_labels).sum().item()
    total = len(query_labels)
    return correct, total

@torch.no_grad() # Decorator ensures no gradients are computed
def evaluate(
    data_loader: DataLoader,
    model_to_evaluate: nn.Module,
    description: str = "Evaluating",
    eval_device: str = EVAL_DEVICE # Default to CPU for final eval
):
    """Evaluates the model on the tasks provided by the data loader."""
    total_predictions = 0
    correct_predictions = 0

    # --- IMPORTANT: Set model to eval mode and move to evaluation device ---
    model_to_evaluate.eval()
    model_to_evaluate.to(eval_device)
    # ---

    with tqdm(data_loader, desc=description, total=len(data_loader)) as tqdm_eval:
        for support_images, support_labels, query_images, query_labels, _ in tqdm_eval:
            correct, total = evaluate_on_one_task(
                model_to_evaluate, support_images, support_labels, query_images, query_labels, eval_device=eval_device
            )
            total_predictions += total
            correct_predictions += correct

            # Update progress bar with running accuracy
            if total_predictions > 0:
                current_acc = 100.0 * correct_predictions / total_predictions
                tqdm_eval.set_postfix(acc=f"{current_acc:.2f}%")

    # Calculate final accuracy
    accuracy = 100.0 * correct_predictions / total_predictions if total_predictions > 0 else 0.0
    print(f"{description} complete. Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_predictions}) on {eval_device}")
    return accuracy

# --- Training Function (for one episode/task) ---
def fit_one_task(
    model_to_train: nn.Module,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
    train_device: str, # Explicit device for training
) -> float:
    """Performs one training step (fits one task) during QAT."""
    optimizer.zero_grad()
    # --- IMPORTANT: Ensure model is in train mode for QAT ---
    model_to_train.train()
    # ---

    # Move data to training device
    support_images = support_images.to(train_device)
    support_labels = support_labels.to(train_device)
    query_images = query_images.to(train_device)
    query_labels = query_labels.to(train_device)

    # Forward pass - Model handles Q/DQ stubs internally during training
    classification_scores = model_to_train(support_images, support_labels, query_images)

    # Calculate loss and update weights
    loss = criterion(classification_scores, query_labels)
    loss.backward()
    optimizer.step()

    return loss.item()

# --- Helper function to create and prepare a model instance for QAT ---
def create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device=QAT_DEVICE):
    """
    Creates a Prototypical Network with a quantization-ready ResNet18 backbone,
    loads optional pre-trained FSL weights, and prepares it for QAT.
    """
    print("\n--- Creating and Preparing Model for QAT ---")
    # 1. Create Quantization-Ready Backbone
    #    Use torchvision.models.quantization.resnet18
    #    quantize=False indicates it's ready for QAT, not static quantization.
    #    Load standard ImageNet weights as a starting point.
    print("Creating quantization-ready ResNet18 backbone (quantize=False)...")
    qat_backbone = quantized_resnet18(weights='IMAGENET1K_V1', quantize=False)
    # Replace the final fully connected layer (classifier) with Identity,
    # as we only need the features before the classification head.
    qat_backbone.fc = nn.Identity()

    # 2. Create the Full Prototypical Network Model
    print("Creating PrototypicalNetworks model with QAT backbone...")
    # Note: Don't move to device yet, weights loading might specify map_location
    model = PrototypicalNetworks(qat_backbone)

    # 3. Load Pre-trained FSL Weights (if provided and exist)
    if pretrained_fsl_weights_path and os.path.exists(pretrained_fsl_weights_path):
        try:
            print(f"Loading FSL state dict from: {pretrained_fsl_weights_path}")
            # Load state dict to CPU first for flexibility
            state_dict = torch.load(pretrained_fsl_weights_path, map_location='cpu')

            # --- Critical Step for loading weights into QAT model ---
            # We need strict=False because:
            #   a) The original model (likely standard ResNet) doesn't have Quant/DeQuant stubs.
            #   b) The backbone module keys might differ slightly if the original wasn't
            #      explicitly using the `torchvision.models.quantization` variant.
            # Filter warnings related to unexpected keys if they occur during loading
            with warnings.catch_warnings():
                 warnings.simplefilter("ignore", category=UserWarning) # Often warns about size mismatches if fc differs
                 model.load_state_dict(state_dict, strict=False)
            print("Pre-trained FSL weights loaded successfully into QAT structure (strict=False).")
            # ---
        except Exception as e:
            print(f"Warning: Error loading pre-trained FSL weights: {e}. Check compatibility. Using ImageNet weights only for backbone.")
    elif pretrained_fsl_weights_path:
        print(f"Warning: Pre-trained FSL weights file not found at {pretrained_fsl_weights_path}. Using ImageNet weights only for backbone.")
    else:
        print("No FSL pre-trained weights path provided. Using ImageNet weights only for backbone.")

    # 4. Move model to the designated QAT device *after* loading weights
    model.to(qat_device)

    # 5. Configure and Prepare for QAT
    model.train() # QAT preparation requires model to be in training mode

    # Select backend ('fbgemm' for x86, 'qnnpack' for ARM). Default usually works.
    # Using get_default_qat_qconfig is generally recommended.
    backend = "qnnpack" if torch.backends.quantized.engine == "qnnpack" else "fbgemm"
    print(f"Using quantization backend: {backend}")
    model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
    print(f"Applied default QAT qconfig for backend {backend}.")

    # --- Fusion Step (Generally NOT needed for torchvision.models.quantization) ---
    # The quantized ResNet variants are typically pre-fused or structured correctly.
    # Explicit fusion here might be redundant or cause issues.
    # print("Skipping explicit fusion step; relying on pre-structured quantized ResNet.")
    # ---

    # 6. Prepare the model for QAT
    # This inserts observers into the model to collect activation statistics during training.
    print("Applying torch.quantization.prepare_qat...")
    # Ensure model is in training mode *before* calling prepare_qat
    model.train()
    # inplace=True modifies the model directly
    torch.quantization.prepare_qat(model, inplace=True)
    print("Model prepared successfully for QAT.")

    return model

# --- Main Execution Logic ---
def main():
    # 1. Download Pre-trained FSL Weights (Optional but recommended)
    if not os.path.exists(PRETRAINED_FSL_WEIGHTS_FILE):
        print(f"Downloading pre-trained FSL weights to {PRETRAINED_FSL_WEIGHTS_FILE}...")
        try:
            # Use wget or curl via subprocess
            subprocess.run(["wget", "-O", PRETRAINED_FSL_WEIGHTS_FILE, PRETRAINED_FSL_WEIGHTS_URL], check=True, timeout=120)
            print("Download complete.")
        except FileNotFoundError:
             print("Error: 'wget' command not found. Please download the weights manually:")
             print(f"URL: {PRETRAINED_FSL_WEIGHTS_URL}")
             print(f"Save as: {PRETRAINED_FSL_WEIGHTS_FILE}")
             return # Exit if weights are needed but download fails
        except subprocess.CalledProcessError as e:
            print(f"Error during download (wget returned non-zero exit status {e.returncode}).")
            return
        except subprocess.TimeoutExpired:
            print("Error: Download timed out.")
            return
        except Exception as e:
            print(f"An unexpected error occurred during download: {e}")
            return
    else:
        print(f"Pre-trained FSL weights file found: {PRETRAINED_FSL_WEIGHTS_FILE}")
    fsl_weights_path = PRETRAINED_FSL_WEIGHTS_FILE if os.path.exists(PRETRAINED_FSL_WEIGHTS_FILE) else None

    # --- Variables to store accuracies and losses for plotting ---
    fp32_acc = 0.0
    qat_pre_conversion_acc = 0.0
    int8_acc = 0.0
    int8_reloaded_acc = 0.0
    training_losses = []


    # --- Optional: Evaluate Original FP32 Model (for baseline comparison) ---
    print("\n--- Evaluating Original FP32 Model (Reference) ---")
    try:
        # Create standard ResNet18 backbone
        ref_backbone = standard_resnet18(weights='IMAGENET1K_V1')
        ref_backbone.fc = nn.Identity()
        ref_model = PrototypicalNetworks(ref_backbone)

        if fsl_weights_path:
            print(f"Loading FSL weights into FP32 reference model...")
            ref_state_dict = torch.load(fsl_weights_path, map_location='cpu')
            # Use strict=False here too, as the PrototypicalNetworks wrapper adds quant/dequant
            # even if the backbone itself is standard. The keys won't perfectly match.
            with warnings.catch_warnings():
                 warnings.simplefilter("ignore", category=UserWarning)
                 ref_model.load_state_dict(ref_state_dict, strict=False)
            print("Loaded FSL weights into reference model.")
        else:
             print("Skipping FSL weight loading for reference model (file not found/download failed).")

        ref_model.to(DEVICE) # Evaluate reference model on the primary device (GPU if available)
        fp32_acc = evaluate(test_loader, ref_model, description="FP32 Reference Eval", eval_device=DEVICE)

        # Clean up GPU memory if used
        del ref_model
        del ref_backbone
        if DEVICE == 'cuda': torch.cuda.empty_cache()

    except Exception as e:
        print(f"Could not evaluate reference FP32 model: {e}")
        # Ensure cleanup even on error
        if 'ref_model' in locals(): del ref_model
        if 'ref_backbone' in locals(): del ref_backbone
        if DEVICE == 'cuda': torch.cuda.empty_cache()
    # --- End Optional FP32 Eval ---


    # 2. Create and Prepare Model for QAT
    qat_model = create_and_prepare_qat_model(pretrained_fsl_weights_path=fsl_weights_path, qat_device=QAT_DEVICE)

    # 3. Perform Quantization Aware Training (Fine-tuning)
    print("\n--- Starting Quantization Aware Training (Fine-tuning) ---")
    # Use a smaller learning rate for fine-tuning QAT
    optimizer = optim.Adam(qat_model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    all_loss = []
    qat_model.to(QAT_DEVICE) # Ensure model is on the correct device for training

    with tqdm(train_loader, total=len(train_loader), desc="QAT Training") as tqdm_train:
        for episode_index, (support_images, support_labels, query_images, query_labels, _) in enumerate(tqdm_train):
            loss_value = fit_one_task(
                qat_model, optimizer, criterion,
                support_images, support_labels, query_images, query_labels,
                train_device=QAT_DEVICE
            )
            all_loss.append(loss_value)
            training_losses.append(loss_value) # Store loss for plotting

            # Log average loss periodically
            if episode_index % LOG_UPDATE_FREQUENCY == 0 and episode_index > 0:
                 avg_interval = min(len(all_loss), LOG_UPDATE_FREQUENCY * 2) # Use available history
                 if avg_interval > 0:
                    avg_loss = sum(all_loss[-avg_interval:]) / avg_interval
                    tqdm_train.set_postfix(loss=f"{avg_loss:.4f}")

    print("QAT Fine-tuning finished.")

    # --- Optional: Evaluate QAT model *before* conversion ---
    # This evaluates the model with observers active, still using QAT device
    print("\n--- Evaluating QAT Model (Before Conversion) ---")
    qat_pre_conversion_acc = evaluate(test_loader, qat_model, description="QAT Pre-Conversion Eval", eval_device=QAT_DEVICE)
    # ---

    # 4. Save the QAT Model State (Weights + Observers)
    print(f"\n--- Saving QAT model state (including observers) to {QAT_STATE_DICT_FILENAME} ---")
    # Important: Save the state dict while the model includes observers, before conversion.
    # Move to CPU before saving for better compatibility.
    qat_model.eval() # Set to eval mode
    qat_model_state_dict_cpu = qat_model.to('cpu').state_dict()
    torch.save(qat_model_state_dict_cpu, QAT_STATE_DICT_FILENAME)
    print("QAT model state saved.")
    # Clean up GPU memory if QAT was done there
    del qat_model
    if QAT_DEVICE == 'cuda': torch.cuda.empty_cache()


    # 5. Convert the Model to Quantized INT8
    print("\n--- Converting Model to Final Quantized INT8 Format ---")
    # A. Create a fresh instance of the model prepared for QAT (on CPU)
    #    This ensures the architecture exactly matches the one used for QAT.
    #    Do not load FSL weights here; they are part of the QAT state dict.
    model_to_convert = create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device='cpu')

    # B. Load the saved QAT state dict (weights + observers) into this fresh instance
    print(f"Loading saved QAT state from: {QAT_STATE_DICT_FILENAME}")
    model_to_convert.load_state_dict(torch.load(QAT_STATE_DICT_FILENAME, map_location='cpu'))
    print("QAT state loaded successfully into conversion model.")

    # C. Convert the model
    model_to_convert.eval() # Ensure model is in evaluation mode for conversion
    model_to_convert.to('cpu') # Conversion typically happens on CPU
    print("Applying torch.quantization.convert...")
    # inplace=False creates a new converted model (safer)
    quantized_int8_model = torch.quantization.convert(model_to_convert, inplace=False)
    print("Model successfully converted to INT8.")
    # Clean up the pre-conversion model
    del model_to_convert
    torch.cuda.empty_cache() # Just in case

    # 6. Save the Final Quantized INT8 Model State Dict
    print(f"\n--- Saving final INT8 quantized model state dict to {FINAL_INT8_STATE_DICT_FILENAME} ---")
    quantized_int8_model.eval() # Ensure eval mode
    # Already on CPU from conversion step
    torch.save(quantized_int8_model.state_dict(), FINAL_INT8_STATE_DICT_FILENAME)
    print("Final INT8 model state dict saved.")


    # 7. Evaluate the Final Quantized INT8 Model (on CPU)
    print("\n--- Evaluating Final Quantized INT8 Model ---")
    int8_acc = evaluate(test_loader, quantized_int8_model, description="Final INT8 Quantized Eval", eval_device=EVAL_DEVICE) # EVAL_DEVICE is CPU


    # --- How to load the final INT8 model later ---
    print("\n--- Example: How to load and use the final INT8 model later ---")
    # 1. Create the *quantized* model architecture instance.
    #    Start with the QAT-ready structure, as the saved state dict keys match that.
    print("Creating base QAT-ready structure (on CPU)...")
    final_model_structure = create_and_prepare_qat_model(pretrained_fsl_weights_path=None, qat_device='cpu')
    final_model_structure.eval()
    # 2. Convert this structure to INT8 *before* loading the state dict.
    #    This makes its layers expect quantized weights/biases.
    print("Converting empty structure to INT8...")
    final_model_quantized_empty = torch.quantization.convert(final_model_structure, inplace=False)
    # 3. Load the saved INT8 state dict.
    print(f"Loading saved INT8 state from: {FINAL_INT8_STATE_DICT_FILENAME}")
    final_model_quantized_empty.load_state_dict(torch.load(FINAL_INT8_STATE_DICT_FILENAME, map_location='cpu'))
    print("Final INT8 state loaded.")
    # 4. The model `final_model_quantized_empty` is now ready for inference on CPU.
    # Example: Evaluate it again to confirm loading worked
    int8_reloaded_acc = evaluate(test_loader, final_model_quantized_empty, description="Reloaded Final INT8 Eval", eval_device=EVAL_DEVICE)
    # ---


    print("\nScript finished successfully.")

    # --- Plotting Code ---
    print("\n--- Generating Plots ---")
    # 1. Accuracy Comparison Bar Chart
    accuracies = [fp32_acc, qat_pre_conversion_acc, int8_acc, int8_reloaded_acc]
    model_names = ['FP32 Ref', 'QAT Pre-Conv', 'INT8', 'INT8 Reloaded']
    plt.figure(figsize=(8, 6))
    plt.bar(model_names, accuracies, color=['blue', 'green', 'red', 'purple'])
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy Comparison of Different Model Stages')
    plt.ylim([min(accuracies)-1, max(accuracies)+1]) # Adjust y-axis limits to data range
    plt.savefig(os.path.join(GRAPHS_DIR, "accuracy_comparison.png"))
    plt.close()
    print(f"Accuracy comparison plot saved to {GRAPHS_DIR}/accuracy_comparison.png")

    # 2. Training Loss Curve
    plt.figure(figsize=(10, 6))
    plt.plot(training_losses)
    plt.xlabel('Training Episodes')
    plt.ylabel('Loss')
    plt.title('QAT Training Loss Curve')
    plt.grid(True)
    plt.savefig(os.path.join(GRAPHS_DIR, "qat_training_loss.png"))
    plt.close()
    print(f"QAT Training Loss plot saved to {GRAPHS_DIR}/qat_training_loss.png")
    print("--- Plots generated successfully in the 'graphs' directory ---")


if __name__ == "__main__":
    main()

Using QAT device: cuda
Using final evaluation device: cpu
Loading Omniglot dataset...
Setting up data loaders...
Data loading complete.
Pre-trained FSL weights file found: ./models/resnet18_with_pretraining.tar

--- Evaluating Original FP32 Model (Reference) ---


<ipython-input-22-39bd8801c7c1>:434: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ref_state_dict = torch.load(fsl_weights_path, map_location='cpu')


Loading FSL weights into FP32 reference model...
Loaded FSL weights into reference model.


FP32 Reference Eval: 100%|██████████| 500/500 [00:15<00:00, 31.58it/s, acc=96.50%]


FP32 Reference Eval complete. Accuracy: 96.50% (24124/25000) on cuda

--- Creating and Preparing Model for QAT ---
Creating quantization-ready ResNet18 backbone (quantize=False)...
Creating PrototypicalNetworks model with QAT backbone...
Loading FSL state dict from: ./models/resnet18_with_pretraining.tar
Pre-trained FSL weights loaded successfully into QAT structure (strict=False).
Using quantization backend: fbgemm
Applied default QAT qconfig for backend fbgemm.
Applying torch.quantization.prepare_qat...


...

Final INT8 Quantized Eval complete. Accuracy: 97.58% (24395/25000) on cpu

--- Example: How to load and use the final INT8 model later ---
Creating base QAT-ready structure (on CPU)...

--- Creating and Preparing Model for QAT ---
Creating quantization-ready ResNet18 backbone (quantize=False)...
Creating PrototypicalNetworks model with QAT backbone...
No FSL pre-trained weights path provided. Using ImageNet weights only for backbone.
Using quantization backend: fbgemm
Applied default QAT qconfig for backend fbgemm.
Applying torch.quantization.prepare_qat...
Model prepared successfully for QAT.
Converting empty structure to INT8...


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/utils.py:407: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
<ipython-input-22-39bd8801c7c1>:560: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loade

Loading saved INT8 state from: ./models/final_int8_proto_omniglot_state_1743851529.pth
Final INT8 state loaded.


Reloaded Final INT8 Eval: 100%|██████████| 500/500 [00:25<00:00, 19.38it/s, acc=97.25%]


Reloaded Final INT8 Eval complete. Accuracy: 97.25% (24313/25000) on cpu

Script finished successfully.

--- Generating Plots ---
Accuracy comparison plot saved to ./graphs/accuracy_comparison.png
QAT Training Loss plot saved to ./graphs/qat_training_loss.png
--- Plots generated successfully in the 'graphs' directory ---
